In [7]:
using CSV, DataFrames, DataFramesMeta, HypothesisTests
PacHypoTbl=DataFrame(CSV.File("TempData/.csv"))


,rat,region,behavior_type,agent_type,trial_exp_num,trial_fam,fband
,String7,String15,String15,String7,String7,String15,String15
1,RRSD17,MOB,Rear,missing,Any[],missing,low_gamma
2,RRSD17,Amygdala,Rear,missing,Any[],missing,low_gamma
3,RRSD17,Ca2,Rear,missing,Any[],missing,low_gamma
4,RRSD17,Ca2,Groom,missing,Any[],missing,low_gamma
5,RRSD17,Amygdala,Groom,missing,Any[],missing,low_gamma
6,RRSD17,MOB,Groom,missing,Any[],missing,low_gamma
7,RRSD17,MOB,Groom,missing,Any[],missing,low_gamma
8,RRSD17,Ca2,Groom,missing,Any[],missing,low_gamma
9,RRSD17,Amygdala,Groom,missing,Any[],missing,low_gamma


Test for Non-Zero PAC 

In [13]:
subs = @subset(PacHypoTbl, :fband .== "low_gamma", :behavior_type .== "Rear", :agent_type .== "Rat", :region .== "Ca2")
test=OneSampleTTest(subs.theta.-subs.null_mag)

One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         0
    point estimate:          6.37984e-5
    95% confidence interval: (5.094e-5, 7.666e-5)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-14

Details:
    number of observations:   79
    t-statistic:              9.875938607034245
    degrees of freedom:       78
    empirical standard error: 6.459978666344884e-6


In [14]:
#show p value
pvalue(test)

2.1849804931313863e-15

In [17]:
#Batch Tests
gdf = groupby(PacHypoTbl, [:fband, :behavior_type, :agent_type, :region])
PvalTbl = @combine(gdf, :pval = pvalue(OneSampleTTest(:mag-:null_mag)))


,fband,behavior_type,agent_type,region,pval
,String15,String15,String7,String15,Float64
1,low_gamma,Rear,missing,MOB,1.46871e-65
2,low_gamma,Rear,missing,Amygdala,1.69282e-87
3,low_gamma,Rear,missing,Ca2,2.91886e-87
4,low_gamma,Rear,missing,Insula,3.41427e-90
5,low_gamma,Rear,Object,MOB,0.000203448
6,low_gamma,Rear,Object,Amygdala,5.20118e-11
7,low_gamma,Rear,Object,Ca2,0.00118253
8,low_gamma,Rear,Object,Insula,2.68359e-6
9,low_gamma,Rear,Robot,MOB,1.36989e-12


Pairwise Comparisons

In [27]:
subs1 = @subset(PacHypoTbl, :fband .== "high_gamma", :behavior_type .== "Immobility", :agent_type .== "Robot", :region .== "MOB", :trial_fam .== "Novel")
subs2 = @subset(PacHypoTbl, :fband .== "high_gamma", :behavior_type .== "Immobility", :agent_type .== "Robot", :region .== "MOB", :trial_fam .== "Familiar")
test = UnequalVarianceTTest(subs1.mag, subs2.mag)

Two sample t-test (unequal variance)
------------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          0.000984807
    95% confidence interval: (-6.036e-5, 0.00203)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.0617

Details:
    number of observations:   [8,71]
    t-statistic:              2.1510721641436312
    degrees of freedom:       8.492814468388683
    empirical standard error: 0.0004578214580914556


In [28]:
pvalue(test)

0.06171428702630704